In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [30]:
web_info_df = pd.read_excel("/Users/ljohnson/repos/livjab/spoofmail/website_privacy.xlsx", sheet_name="Sheet1")
web_info_df.head()

,Country_Rank,Website,Trustworthiness,Avg_Daily_Visitors,Child_Safety,Avg_Daily_Pageviews,Privacy,Traffic_Rank,Daily_Pageviews,Daily_Pageviews_per_user_percentage,Location,Hosted_by,Registrant,Registrar,country,Privacy_policy
0,1,www.google.com,Excellent,518 108 189,Excellent,4 186 314 171,Excellent,1,8.448960,33.9,United States,Google Inc.,DNS Admin (Google Inc.),MARKMONITOR INC.,United States,"Privacy Policy Last modified: December 18, 201..."
1,2,www.youtube.com,Excellent,506 457 282,Excellent,2 679 159 025,Excellent,2,5.934720,3.8,United States,Google Inc.,DNS Admin (Google Inc.),MARKMONITOR INC.,United States,"Last modified: June 24, 2013 (view archived ve..."
2,3,www.facebook.com,Excellent,270 071 255,Good,1 082 985 733,Excellent,3,2.046720,-8.2,Ireland,Facebook,"Domain Administrator (Facebook, Inc.)",MARKMONITOR INC.,United States,Data Policy\nThis policy describes the informa...
3,4,www.reddit.com,Excellent,60 614 015,Excellent,649 176 107,Excellent,9,1.390080,0.8,United States,Fastly,Domain Administrator (Reddit Inc),GANDI SAS,United States,"Reddit Privacy Policy\nEffective June 8, 2018...."
4,5,www.amazon.com,Excellent,48 820 414,Excellent,399 350 988,Excellent,11,0.770688,-1.5,United States,Amazon Technologies Inc.,"Hostmaster, Amazon Legal Dept. (Amazon Technol...",MARKMONITOR INC.,United States,Amazon Privacy Notice\n\nLast updated: August ...


In [31]:
web_info_df["Privacy"].value_counts()

Excellent         46
Poor               1
Unsatisfactory     1
Good               1
Name: Privacy, dtype: int64

In [32]:
# target will be privacy column
# will change to 1/0 (excellent/not excellent)

web_info_df["Privacy"] = web_info_df["Privacy"].replace({"Excellent": 1, "Good": 0, "Unsatisfactory": 0, "Poor": 0})
web_info_df["Privacy"].value_counts()

1    46
0     3
Name: Privacy, dtype: int64

In [33]:
# getting privacy policy text tokenized

docs = web_info_df["Privacy_policy"].values.tolist()

In [18]:
import gensim
from gensim.utils import simple_preprocess

def tokenize(doc):
    for doc in docs:
        yield(simple_preprocess(str(doc), deacc=True))

In [20]:
data_words = list(tokenize(docs))
data_words[0][:10]

['privacy',
 'policy',
 'last',
 'modified',
 'december',
 'view',
 'archived',
 'versions',
 'the',
 'hyperlinked']

In [21]:
from nltk.corpus import stopwords

stop_words = stopwords.words('english')

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data_words = remove_stopwords(data_words)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', tokenizer=lambda doc: doc, lowercase=False)

dtm = tfidf.fit_transform(data_words)
df = pd.DataFrame(dtm.todense(), columns = tfidf.get_feature_names())
df.head()

/Users/ljohnson/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,abandoning,abbot,abide,abides,ability,able,abn,aboutads,aboutcookies,absolute,...,youradchoices,youtube,yume,zealand,zillow,zip,zone,zoosk,zumobi,zune
0,0.0,0.0,0.0,0.0,0.006094,0.008222,0.0,0.0,0.0,0.0,...,0.0,0.084656,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.011678,0.015757,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.006170,0.008325,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.009838,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.021480,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.008143,0.016479,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.012983,0.0,0.0,0.0


In [23]:
from sklearn.neighbors import NearestNeighbors

nn = NearestNeighbors(n_neighbors=10, algorithm='ball_tree')

nn.fit(dtm.todense())

NearestNeighbors(algorithm='ball_tree', leaf_size=30, metric='minkowski',
         metric_params=None, n_jobs=None, n_neighbors=10, p=2, radius=1.0)

In [24]:
sample_policy = ["We will sell your data to third parties. Nothing is secure. We do not care about your privacy"]

In [25]:
new = tfidf.transform(sample_policy)

nn.kneighbors(new.todense())

(array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]]),
 array([[34, 27, 12, 31, 20, 38, 33,  1,  4, 30]]))

In [34]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

vect = TfidfVectorizer(stop_words='english', tokenizer=lambda doc: doc, lowercase=False)
sgdc = SGDClassifier()

pipe = Pipeline([('vect', vect), ('clf', sgdc)])

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'clf__max_iter':(20, 10, 100)
}

grid_search = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1, verbose=1)

grid_search.fit(data_words, web_info_df["Privacy"])

Fitting 5 folds for each of 9 candidates, totalling 45 fits


/Users/ljohnson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:   17.2s finished
/Users/ljohnson/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)
/Users/ljohnson/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['b', 'c', 'd', 'e', 'f', 'g', 'h', 'k',

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...m_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'clf__max_iter': (20, 10, 100)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [35]:
y_pred = grid_search.predict(sample_policy)
y_pred

array([1])